In [ ]:
import pandas as pd
import sys
import os
import numpy as np
import cupy as cp
import sklearn
import scipy
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.kernel_ridge import KernelRidge
from timeit import default_timer as timer 
from sys import getsizeof
from matplotlib import pyplot as plt
import cupyx
np.set_printoptions(suppress=True)

In [ ]:
#def pivoted_cholesky_cpu(matrix, max_rank, max_error_tolerance=0.001):


#Pivoted_cholesky_cpu : to compute PCD in CPU 
#parameters taken: positive semi definite matrix, maximum rank of the approximation required, maximum error tolerance
#returns list : [approximated matrix LL^T, rank_list [1...max_rank], norm list containing norm LL^T for all ranks, time taken for each rank in CPU] 

In [ ]:
#[1]: H Harbrecht, M Peters, R Schneider. On the low-rank approximation by the pivoted Cholesky decomposition. Applied numerical mathematics, 62(4):428-440, 2012.
#reference : https://github.com/tensorflow/probability/blob/v0.12.2/tensorflow_probability/python/math/linalg.py#L264-L404

def pivoted_cholesky_cpu(matrix, max_rank, max_error_tolerance=1e-3):
    
    #initialize result lists
    norm_array = [] 
    rank_array = []
    time_array = []
    
    if max_rank > np.linalg.matrix_rank(matrix): # input max rank should not exceed full rank of original matrix
        max_rank = np.linalg.matrix_rank(matrix)
    
    matrix_shape = np.asarray(matrix.shape)
    matrix_diag = np.diagonal(matrix)
    orig_error = np.amax(matrix_diag)
    
    Error_array = []
    norm_array = []
    
    start = timer()
    
    def body(m, pchol, perm, matrix_diag):
        
        maxi = np.argmax(matrix_diag[perm[m:]]) + m  # taking index of highest diagonal element

        maxval = matrix_diag[perm][maxi] # value of highest diagonal element
        perm[m], perm[maxi] = perm[maxi], perm[m]  

        row = matrix[perm[m]][perm[m + 1:]] 

        
        def batch_gather(params, indices, axis=-1):
            return np.take(params, indices, axis=axis)
        
        prev_rows = pchol[..., :m, :]
        prev_rows_perm_m_onward = batch_gather(prev_rows, perm[..., m + 1:])
        prev_rows_pivot_col = batch_gather(prev_rows, perm[..., m:m + 1])
        
        row -= np.sum(prev_rows_perm_m_onward * prev_rows_pivot_col, axis = -2)

        pivot = np.sqrt(maxval) # worst approximated element used as a pivot
        
        row /= pivot # row elements divided by pivot

        row = np.concatenate([[pivot], row], -1)
        
        matrix_diag_copy = matrix_diag.copy()
        matrix_diag_copy[perm[m:]] -= row**2
        
        pchol[m, perm[m:]] = row # adding row to pchol
        pchol_shape = pchol.shape

        return m + 1, pchol, perm, matrix_diag_copy

    m = np.int64(0)
    pchol = np.zeros(matrix_shape, dtype=matrix.dtype)[..., :max_rank, :]
    perm = np.broadcast_to(
        range(matrix_shape[-1]), matrix_shape[:-1])
    perm = perm.copy()

    def check_cond(m, pchol, perm, matrix_diag):
        del pchol
        del perm
        error = np.linalg.norm(matrix_diag, ord=1, axis=-1)
        max_error = np.amax(error / orig_error)
        return (m >= max_rank) |( max_error <= max_error_tolerance) # loop ends if m exceeds max rank or error is less than max allowed error tolerance

    while(1): #Each iteration rank is increased (one more row is added so that it makes a better approximation)
        m, pchol, perm, matrix_diag = body(m, pchol, perm, matrix_diag)
        pchol_T = np.transpose(pchol)
        lr_approx = np.matmul(pchol_T, pchol)
        time_array.append(timer() - start) 
        rank_array.append(np.linalg.matrix_rank(lr_approx))
        norm_array.append(np.linalg.norm(lr_approx))
        if (m % 50 == 0):
            print(" computing... rank {} approximation completed (CPU) \n".format(m))
        if check_cond(m, pchol, perm, matrix_diag):
            break
    
    print("PCD Computation completed \n")
    return lr_approx, rank_array, norm_array, time_array


In [ ]:

#Pivoted_cholesky_gpu : to compute PCD in GPU 

#returns list : [approximated matrix LL^T, rank_list [1...max_rank], norm list containing norm LL^T for all ranks, time taken for each rank in GPU] 

In [ ]:



def pivoted_cholesky_gpu(matrix, max_rank,  max_error_tolerance=0.001):
    
    norm_array_gpu = []
    rank_array_gpu = []
    time_array_gpu = []

    if max_rank > np.linalg.matrix_rank(matrix):
            max_rank = np.linalg.matrix_rank(matrix)
    
    matrix_shape = np.asarray(matrix.shape)
    matrix_diag = cp.diagonal(matrix)
    orig_error = cp.amax(matrix_diag)
    max_rank = cp.asnumpy(max_rank)
    
    start = timer()

    def body(m, pchol, perm, matrix_diag):

        maxi = np.argmax(matrix_diag[perm[m:]]) + m

        maxval = matrix_diag[perm][maxi]
        perm[m], perm[maxi] = perm[maxi], perm[m]

        row = matrix[perm[m]][perm[m + 1:]]

        # row = cp.array(row)
        pchol = cp.asnumpy(pchol)

        
        def batch_gather(params, indices, axis=-1):
            return cp.take(params, indices, axis=axis)
        

        prev_rows = pchol[..., :m, :]
        prev_rows_perm_m_onward = batch_gather(prev_rows, perm[..., m + 1:])
        prev_rows_pivot_col = batch_gather(prev_rows, perm[..., m:m + 1])
        
        row -= cp.sum(prev_rows_perm_m_onward * prev_rows_pivot_col, axis = -2)
        pivot = np.sqrt(maxval)

        row /= pivot
        row = np.concatenate(([pivot], row), -1)
        
        matrix_diag_copy = matrix_diag.copy()
        matrix_diag_copy[perm[m:]] -= row**2
        
        pchol[m, perm[m:]] = row
        pchol_shape = pchol.shape

        return m + 1, pchol, perm, matrix_diag_copy

    m = cp.int64(0)
    pchol = cp.zeros(matrix_shape, dtype=matrix.dtype)[..., :max_rank, :]
    perm = np.broadcast_to(
        range(matrix_shape[-1]), matrix_shape[:-1])
    perm = perm.copy()

    def check_cond(m, pchol, perm, matrix_diag):
        del pchol
        del perm
        error = np.linalg.norm(matrix_diag, ord=1, axis=-1)
        max_error = cp.amax(error / orig_error)
        return (m >= max_rank) |( max_error <= max_error_tolerance)

    while(1):
        m, pchol, perm, matrix_diag = body(m, pchol, perm, matrix_diag)
        pchol = cp.array(pchol)
        pchol_T = cp.transpose(pchol)
        lr_approx = cp.matmul(pchol_T, pchol)
        time_array_gpu.append(timer() - start)  
        rank_array_gpu.append(np.linalg.matrix_rank(lr_approx))
        norm_array_gpu.append(np.linalg.norm(lr_approx))
        if (m % 100 == 0):
            print(" computing... rank {} approximation completed (GPU) \n".format(m))
        
        if check_cond(m, pchol, perm, matrix_diag):
            break
            
    print("PCD Computation completed \n")
    return lr_approx, rank_array_gpu, norm_array_gpu, time_array_gpu

In [ ]:
#Pivoted_cholesky. Returns only the low rank matrix 'L'. L*L_transpose would time the approximation
#Use if no time or norm error analysis is needed 

In [ ]:

#[1]: H Harbrecht, M Peters, R Schneider. On the low-rank approximation by the pivoted Cholesky decomposition. Applied numerical mathematics, 62(4):428-440, 2012.
#reference : https://github.com/tensorflow/probability/blob/v0.12.2/tensorflow_probability/python/math/linalg.py#L264-L404


def pivoted_cholesky(matrix, max_rank,  max_error_tolerance=0.001):
    
    if max_rank > np.linalg.matrix_rank(matrix):
            max_rank = np.linalg.matrix_rank(matrix)
    
    matrix_shape = np.asarray(matrix.shape)
    matrix_diag = cp.diagonal(matrix)
    orig_error = cp.amax(matrix_diag)
    max_rank = cp.asnumpy(max_rank)
    
    start = timer()

    def body(m, pchol, perm, matrix_diag):

        maxi = np.argmax(matrix_diag[perm[m:]]) + m

        maxval = matrix_diag[perm][maxi]
        perm[m], perm[maxi] = perm[maxi], perm[m]

        row = matrix[perm[m]][perm[m + 1:]]

        # row = cp.array(row)
        pchol = cp.asnumpy(pchol)

        
        def batch_gather(params, indices, axis=-1):
            return cp.take(params, indices, axis=axis)
        

        prev_rows = pchol[..., :m, :]
        prev_rows_perm_m_onward = batch_gather(prev_rows, perm[..., m + 1:])
        prev_rows_pivot_col = batch_gather(prev_rows, perm[..., m:m + 1])
        
        row -= cp.sum(prev_rows_perm_m_onward * prev_rows_pivot_col, axis = -2)
        pivot = np.sqrt(maxval)

        row /= pivot
        row = np.concatenate(([pivot], row), -1)
        
        matrix_diag_copy = matrix_diag.copy()
        matrix_diag_copy[perm[m:]] -= row**2
        
        pchol[m, perm[m:]] = row
        pchol_shape = pchol.shape

        return m + 1, pchol, perm, matrix_diag_copy

    m = cp.int64(0)
    pchol = cp.zeros(matrix_shape, dtype=matrix.dtype)[..., :max_rank, :]
    perm = np.broadcast_to(
        range(matrix_shape[-1]), matrix_shape[:-1])
    perm = perm.copy()

    def check_cond(m, pchol, perm, matrix_diag):
        del pchol
        del perm
        error = np.linalg.norm(matrix_diag, ord=1, axis=-1)
        max_error = cp.amax(error / orig_error)
        return (m >= max_rank) |( max_error <= max_error_tolerance)

    while(1):
        m, pchol, perm, matrix_diag = body(m, pchol, perm, matrix_diag)
        pchol = cp.array(pchol)
        if (m % 100 == 0):
            print(" computing... ...rank {} approximation completed (GPU) \n".format(m))
        
        if check_cond(m, pchol, perm, matrix_diag):
            break
    
    print("PCD Computation completed \n")
    pchol_T = cp.transpose(pchol)
    return pchol_T

In [ ]:
#Kernel ridge regression implementation 

In [ ]:
def execute_krr(lr, ridge_parameter, X_train, y_train, X_test):

    lr_approx_ridge = lr + cp.identity(len(X_train)) * ridge_parameter # adding ridge parameter
    
    alpha = cp.linalg.solve(lr_approx_ridge, y_train)  # solving for alpha 

    y_prediction = []
    
    X_train, X_test = cp.asnumpy(X_train), cp.asnumpy(X_test)
    for j in range(len(X_test)):
        x_test_val = X_test[j].reshape(1,-1)
        pred = 0
        for i in range(len(X_train)): #computing the predicted values and storing in the prediction vector
            x_train_val = X_train[i].reshape(1,-1)
            similarity = cp.array(sklearn.metrics.pairwise.laplacian_kernel(x_train_val, x_test_val)) # computing similarity between evaluation and training points 
            temp = alpha[i][0] * similarity[0]     
            pred += temp[0]
        y_prediction.append(pred)

    return cp.asarray(y_prediction)

In [ ]:
#load_data

In [ ]:
X = pd.read_csv('energy.csv', usecols=['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8'])
y = pd.read_csv('energy.csv', usecols=['Y1'])

X = X.values
y = y.values

# start_load = timer()

# X = np.load('data/md17_X.npy')

# y = np.load('data/md17_Y.npy')

# print("load data time = ", timer() - start_load)


In [ ]:
#to use only 'n' rows of a large dataset

In [ ]:
number_of_rows = 300 
X = X[:number_of_rows]
y = y[:number_of_rows]

X.shape, y.shape

In [ ]:
#train, test split, #create_kernel_matrix

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) # train - test split - 4:1 (Use 20% for testing, rest for training) 
kernel_matrix = sklearn.metrics.pairwise.laplacian_kernel(X_train, Y=None, gamma=None) # Creating kernel matrix
kernel_matrix.shape

kernel_matrix.shape, np.linalg.norm(kernel_matrix)

In [ ]:
#test for pivoted_cholesky. returns L, compare norm of (LL^T) to norm of original kernel matrix

In [ ]:
lr_approx_gpu = pivoted_cholesky(kernel_matrix, 100, 0) 
len(lr_approx_gpu)

print("norm of Kernel Matrix = ", cp.linalg.norm(kernel_matrix))

norm_approx = cp.linalg.norm(cp.dot(lr_approx_gpu, lr_approx_gpu.T))
                             
print("Rank of L = {}, norm of approximated matrix LL^T = {}".format(np.linalg.matrix_rank(lr_approx_gpu), norm_approx))

In [ ]:
#Execute PCD on GPU
#Get the result list: [LL^T, rank_list, norm_list, time_list]
#function parameters: pivoted_cholesky_gpu(kernel_matrix, max_rank, error_tolerance=0(default))

In [ ]:
start = timer()
pcd_gpu_results = pivoted_cholesky_gpu(kernel_matrix, 2400, 0) 
print("time taken in GPU= ", timer() - start )

In [ ]:
#Execute PCD on CPU
#Get the result list: [LL^T, rank_list, norm_list, time_list]
#function parameters: pivoted_cholesky_gpu(kernel_matrix, max_rank, error_tolerance=0(default))

In [ ]:
start = timer()
pcd_cpu_results = pivoted_cholesky_cpu(kernel_matrix, 2400, 0) 
print("time taken in CPU= ", timer() - start )

In [ ]:
#store cupy array values in separate list for easier plotting 

In [ ]:


lr_approx = pcd_gpu_results[0]


rank_list = [cp.asnumpy(x) for x in pcd_gpu_results[1]]
norm_list = [cp.asnumpy(x) for x in pcd_gpu_results[2]]
time_list = pcd_gpu_results[3]

In [ ]:
#plot results for rank vs error in norm. see specific results

In [ ]:
plt.title("Rank vs norm error") 
plt.xlabel("Rank of approximated matrix") 
plt.ylabel("Error in norm") 

nr = 100 # gap between ranks in results shown

orig_norm = np.linalg.norm(kernel_matrix)
plt.plot(rank_list, orig_norm - norm_list)
plt.show()
# plt.savefig('./figures/768rank_norm.png')
    
print("rank {} error".format("     "))
for n1, n2 in zip(rank_list, norm_list):
    if (n1 < 10) or (n1 % nr == 0): 
        print("{}   &   {:.6f}".format(n1, orig_norm - n2), "   \\\ \hline")

In [ ]:
#plot results for time taken for PCD CPU vs GPU

In [ ]:
plt.title("time taken as rank increases") 
plt.xlabel("rank") 
plt.ylabel("time(s)") 

plt.plot(rank_list, pcd_cpu_results[3], 'r', time_list, 'g')
plt.legend(["time taken in cpu", "time taken in gpu"], loc ="lower right")
plt.savefig('./figures/time2.png')
# plt.show()



# lr_approx[3]

In [ ]:
#plot speed up in gpu increasing rank

In [ ]:
speed_up = []
for i, j in zip(pcd_cpu_results[3], time_list):
    speed_up.append(i/j)
    
plt.title("GPU speed up as matrix size increases") 
plt.xlabel("rank") 
plt.ylabel("speedup") 

plt.plot(rank_list, speed_up, 'g')
plt.savefig('./figures/speedup_2.png')

In [ ]:
np.linalg.norm(pcd_gpu_results[0]), np.linalg.norm(kernel_matrix), np.linalg.matrix_rank(pcd_gpu_results[0]), np.linalg.matrix_rank(kernel_matrix)

In [ ]:
#kernel ridge regression Implementation

In [ ]:
def execute_krr(lr, ridge_parameter, X_train, y_train, X_test):

    lr_approx_ridge = lr + cp.identity(len(X_train)) * ridge_parameter # adding ridge parameter
    
    alpha = cp.linalg.solve(lr_approx_ridge, y_train)  # solving for alpha 

    y_prediction = []
    
    X_train, X_test = cp.asnumpy(X_train), cp.asnumpy(X_test)
    for j in range(len(X_test)):
        x_test_val = X_test[j].reshape(1,-1)
        pred = 0
        for i in range(len(X_train)): #computing the predicted values and storing in the prediction vector
            x_train_val = X_train[i].reshape(1,-1)
            similarity = cp.array(sklearn.metrics.pairwise.laplacian_kernel(x_train_val, x_test_val)) # computing similarity between evaluation and training points 
            temp = alpha[i][0] * similarity[0]     
            pred += temp[0]
        y_prediction.append(pred)

    return cp.asarray(y_prediction)

In [ ]:
#Loop to execute krr for Different rank approximations of the kernel matrix and generate results(RMSE as the generalization error)

In [ ]:


max_rank_values = [100, 200, 250, 300] # values for ranks to experiment on
X_train_cp, y_train_cp, X_test_cp, y_test_cp, kernel_matrix_cp = cp.array(X_train), cp.array(y_train), cp.array(X_test), cp.array(y_test), cp.array(kernel_matrix)

rank_err = []

rank_pred = []

ridge_parameter = 0.001

for rank_value in max_rank_values:
    lr_approx_gpu = pivoted_cholesky_gpu(kernel_matrix, rank_value, 0) 
    y_predarray_gpu = execute_krr(lr_approx_gpu[0], ridge_parameter, X_train_cp, y_train_cp, X_test_cp)  
    
    rmse_krr_manual_gpu = np.sqrt(mean_squared_error(y_test, cp.asnumpy(y_predarray_gpu))) #compute root mean squared error for generalization error of that rank
    
    print("RMSE(root mean squared error) for {} rank approximation = {}".format(rank_value, rmse_krr_manual_gpu, "\n"))
    
    print("____________________________________________________________________________-")
    
    
    rank_pred.append(rmse_krr_manual_gpu) # add rmse



In [ ]:
#"plot" generalization error vs rank with the constant values in max_rank_values

In [ ]:
plt.title("Generalization error per rank") 
plt.xlabel("Rank") 
plt.ylabel("generalization error: rmse") 

plt.plot(max_rank_values, rank_pred)
plt.legend(["ridge parameter = {}".format(ridge_parameter)])
plt.savefig('./figures/rmse.png')
plt.show()